In [1]:
import dask
dask.config.set({"dataframe.query-planning": False})

import scanpy as sc
import spatialdata_io
import numpy as np
import pandas as pd
import sys
import argparse
from pathlib import Path

sys.path.append("../../../workflow/scripts/")
import preprocessing
import readwrite

cfg = readwrite.config()

/work/PRTNR/CHUV/DIR/rgottar1/spatial/env/jbac/miniforge3/envs/spatial/lib/python3.13/site-packages/dask/dataframe/__init__.py:31: FutureWarning: The legacy Dask DataFrame implementation is deprecated and will be removed in a future version. Set the configuration option `dataframe.query-planning` to `True` or None to enable the new Dask Dataframe implementation and silence this warning.
  warnings.warn(
/work/PRTNR/CHUV/DIR/rgottar1/spatial/env/jbac/miniforge3/envs/spatial/lib/python3.13/site-packages/spatialdata/_core/query/relational_query.py:504: FutureWarning: functools.partial will be a method descriptor in future Python versions; wrap it in enum.member() if you want to preserve the old behavior
  left = partial(_left_join_spatialelement_table)
/work/PRTNR/CHUV/DIR/rgottar1/spatial/env/jbac/miniforge3/envs/spatial/lib/python3.13/site-packages/spatialdata/_core/query/relational_query.py:505: FutureWarning: functools.partial will be a method descriptor in future Python versions; wra

# sample

In [2]:
# read counts
adata = readwrite.read_xenium_sample(
    '../../../data/xenium/processed/segmentation/proseg/breast/breast/0OE1/0OE1/raw_results',
    cells_as_circles=False,
    cells_boundaries=False,
    cells_boundaries_layers=False,
    nucleus_boundaries=False,
    cells_labels=False,
    nucleus_labels=False,
    transcripts=False,
    morphology_mip=False,
    morphology_focus=False,
    aligned_images=False,
    anndata=True,
)

# need to round proseg expected counts for resolVI to run
# no need for if statement, doesn't change anything to other segmentation methods
adata.X.data = adata.X.data.astype(np.float32).round()
adata.obs_names = adata.obs_names.astype(str)

/work/PRTNR/CHUV/DIR/rgottar1/spatial/env/jbac/miniforge3/envs/spatial/lib/python3.11/site-packages/anndata/_core/aligned_df.py:68: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


metrics_summary.csv not found at: ../../../data/xenium/processed/segmentation/proseg/breast/breast/0OE1/0OE1/raw_results/metrics_summary.csv


/work/PRTNR/CHUV/DIR/rgottar1/spatial/env/jbac/miniforge3/envs/spatial/lib/python3.11/site-packages/spatialdata/_core/spatialdata.py:185: UserWarning: The table is annotating 'cell_polygons', which is not present in the SpatialData object.
  self.validate_table_in_spatialdata(v)
/work/PRTNR/CHUV/DIR/rgottar1/spatial/env/jbac/miniforge3/envs/spatial/lib/python3.11/site-packages/anndata/_core/anndata.py:822: UserWarning: 
AnnData expects .obs.index to contain strings, but got values like:
    [0, 1, 2, 3, 4]

    Inferred to be: integer

  names = self._prep_dim_index(names, "obs")


In [4]:
import dask
dask.config.set({"dataframe.query-planning": False})

import scanpy as sc
import spatialdata_io
import numpy as np
import pandas as pd
import scvi
import sys
import argparse
from pathlib import Path

sys.path.append("../../../workflow/scripts/")
import preprocessing
import readwrite
cfg = readwrite.config()

# Access the arguments
sample_dir = Path('/work/PRTNR/CHUV/DIR/rgottar1/spatial/env/xenium_paper/data/xenium/processed/segmentation/10x_0um/breast/breast/0OE1/0OE1')
cell_type_labels = Path('/work/PRTNR/CHUV/DIR/rgottar1/spatial/env/xenium_paper/data/xenium/processed/cell_type_annotation/10x_0um/breast/breast/0OE1/0OE1/lognorm/reference_based/matched_reference_combo/rctd_class_aware/Level2/single_cell/labels.parquet')
# cell_type_labels = None
proseg_results_dir = sample_dir / 'raw_results'
xeniumranger_dir = sample_dir / 'normalised_results/outs'


# params from pipeline config
min_counts = 10
min_features = 5
max_counts = float("inf")
max_features = float("inf")
min_cells = 5

mixture_k = 50
max_epochs = 50

adata = readwrite.read_xenium_sample(
    xeniumranger_dir,
    cells_as_circles=False,
    cells_boundaries=False,
    cells_boundaries_layers=False,
    nucleus_boundaries=False,
    cells_labels=False,
    nucleus_labels=False,
    transcripts=False,
    morphology_mip=False,
    morphology_focus=False,
    aligned_images=False,
    anndata=True,
)

# need to round proseg expected counts for resolVI to run
# no need for if statement, doesn't change anything to other segmentation methods
adata.X.data = adata.X.data.astype(np.float32).round()
adata.obs_names = adata.obs_names.astype(str)

if cell_type_labels is not None:
    labels_key = "labels_key"
    semisupervised = True
    adata.obs[labels_key] = pd.read_parquet(cell_type_labels).set_index("cell_id").iloc[:, 0].astype('category')
    adata = adata[adata.obs[labels_key].notna()]
else:
    labels_key = None
    semisupervised = False

# preprocess (QC filters only)
# resolvi requires at least 5 counts in each cell
preprocessing.preprocess(
    adata,
    normalize=False,
    log1p=False,
    scale="none",
    pca=False,
    umap=False,
    save_raw=False,
    min_counts=min_counts,
    min_genes=min_features,
    max_counts=max_counts,
    max_genes=max_features,
    min_cells=min_cells,
    backend="cpu",
)


scvi.external.RESOLVI.setup_anndata(
    adata, labels_key=labels_key, layer=None, prepare_data_kwargs={"spatial_rep": "spatial"}
)
resolvi = scvi.external.RESOLVI(adata, mixture_k=mixture_k, semisupervised=semisupervised)
# resolvi.train(max_epochs=max_epochs)
resolvi.save('model',overwrite=True)

INFO     reading                                                                                                   
         /work/PRTNR/CHUV/DIR/rgottar1/spatial/env/xenium_paper/data/xenium/processed/segmentation/10x_0um/breast/b
         reast/0OE1/0OE1/normalised_results/outs/cell_feature_matrix.h5                                            


/work/PRTNR/CHUV/DIR/rgottar1/spatial/env/jbac/miniforge3/envs/spatial/lib/python3.11/site-packages/spatialdata/_core/spatialdata.py:185: UserWarning: The table is annotating 'cell_labels', which is not present in the SpatialData object.
  self.validate_table_in_spatialdata(v)
/work/PRTNR/CHUV/DIR/rgottar1/spatial/env/jbac/miniforge3/envs/spatial/lib/python3.11/site-packages/scanpy/preprocessing/_simple.py:284: ImplicitModificationWarning: Trying to modify attribute `.var` of view, initializing view as actual.
  adata.var["n_cells"] = number


Removed 0  cells...
Removed 0  genes...


/work/PRTNR/CHUV/DIR/rgottar1/spatial/env/jbac/miniforge3/envs/spatial/lib/python3.11/site-packages/scanpy/neighbors/__init__.py:700: FutureWarning: `transformer='rapids'` is deprecated. Use `rapids_singlecell.tl.neighbors` instead.
  warn(msg, FutureWarning)
/work/PRTNR/CHUV/DIR/rgottar1/spatial/env/jbac/miniforge3/envs/spatial/lib/python3.11/site-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(


INFO     Generating sequential column names                                                                        
INFO     Generating sequential column names                                                                        


/work/PRTNR/CHUV/DIR/rgottar1/spatial/env/jbac/miniforge3/envs/spatial/lib/python3.11/site-packages/scvi/data/fields/_dataframe_field.py:187: UserWarning: Category 0 in adata.obs['_scvi_ind_x'] has fewer than 3 cells. Models may not train properly.
  categorical_mapping = _make_column_categorical(


In [13]:
import dask
dask.config.set({"dataframe.query-planning": False})

import spatialdata_io
import numpy as np
import pandas as pd
import scvi
import sys
import argparse
from pathlib import Path

sys.path.append("../../../workflow/scripts/")
import preprocessing
import readwrite
cfg = readwrite.config()

# Access the arguments
sample_dir = Path('/work/PRTNR/CHUV/DIR/rgottar1/spatial/env/xenium_paper/data/xenium/processed/segmentation/10x_0um/breast/breast/0OE1/0OE1')
# cell_type_labels = Path('/work/PRTNR/CHUV/DIR/rgottar1/spatial/env/xenium_paper/data/xenium/processed/cell_type_annotation/10x_0um/breast/breast/0OE1/0OE1/lognorm/reference_based/matched_reference_combo/rctd_class_aware/Level2/single_cell/labels.parquet')
cell_type_labels = None
proseg_results_dir = sample_dir / 'raw_results'
xeniumranger_dir = sample_dir / 'normalised_results/outs'
dir_resolvi_model= '/work/PRTNR/CHUV/DIR/rgottar1/spatial/env/xenium_paper/results/resolvi/10x_0um/breast/breast/0OE1/0OE1/mixture_k=50/model'
dir_resolvi_model_supervised = '/work/PRTNR/CHUV/DIR/rgottar1/spatial/env/xenium_paper/results/resolvi_supervised/10x_0um/breast/breast/0OE1/0OE1/lognorm/model'

# dir_resolvi_model='model'

# params from pipeline config
min_counts = 10
min_features = 5
max_counts = float("inf")
max_features = float("inf")
min_cells = 5

mixture_k = 50
max_epochs = 50


num_samples=30
batch_size=1000

adata = readwrite.read_xenium_sample(
    xeniumranger_dir,
    cells_as_circles=False,
    cells_boundaries=False,
    cells_boundaries_layers=False,
    nucleus_boundaries=False,
    cells_labels=False,
    nucleus_labels=False,
    transcripts=False,
    morphology_mip=False,
    morphology_focus=False,
    aligned_images=False,
    anndata=True,
)

# need to round proseg expected counts for resolVI to run
# no need for if statement, doesn't change anything to other segmentation methods
adata.X.data = adata.X.data.astype(np.float32).round()
adata.obs_names = adata.obs_names.astype(str)

if cell_type_labels is not None:
    labels_key = "labels_key"
    semisupervised = True
    adata.obs[labels_key] = pd.read_parquet(cell_type_labels).set_index("cell_id").iloc[:, 0].astype('category')
    adata = adata[adata.obs[labels_key].notna()]
else:
    labels_key = None
    semisupervised = False

# preprocess (QC filters only)
# resolvi requires at least 5 counts in each cell
preprocessing.preprocess(
    adata,
    normalize=False,
    log1p=False,
    scale="none",
    pca=False,
    umap=False,
    save_raw=False,
    min_counts=min_counts,
    min_genes=min_features,
    max_counts=max_counts,
    max_genes=max_features,
    min_cells=min_cells,
    backend="cpu",
)



resolvi = scvi.external.RESOLVI.load(dir_resolvi_model, adata=adata)

samples_corr = resolvi.sample_posterior(
    model=resolvi.module.model_corrected,
    return_sites=["obs"],
    return_observed=True,
    summary_fun={"post_donor_q50": np.median},
    num_samples=num_samples,
    batch_size=batch_size,
    summary_frequency=100,
)
samples_corr = pd.DataFrame(samples_corr).T

samples = resolvi.sample_posterior(
    model=resolvi.module.model_residuals,
    return_sites=["mixture_proportions"],
    summary_fun={"post_donor_means": np.mean},
    num_samples=num_samples,
    batch_size=batch_size,
    summary_frequency=100,
)
samples_proportions = pd.DataFrame(samples).T

### save
samples_corr = pd.DataFrame(
    samples_corr.loc["post_donor_q50", "obs"],
    index=adata.obs_names,
    columns=adata.var_names,
)
samples_proportions = pd.DataFrame(
    samples_proportions.loc["post_donor_means", "mixture_proportions"],
    index=adata.obs_names,
    columns=["true_proportion", "diffusion_proportion", "background_proportion"],
)


INFO     reading                                                                                                   
         /work/PRTNR/CHUV/DIR/rgottar1/spatial/env/xenium_paper/data/xenium/processed/segmentation/10x_0um/breast/b
         reast/0OE1/0OE1/normalised_results/outs/cell_feature_matrix.h5                                            
Removed 11449  cells...
Removed 0  genes...
INFO     File                                                                                                      
         /work/PRTNR/CHUV/DIR/rgottar1/spatial/env/xenium_paper/results/resolvi/10x_0um/breast/breast/0OE1/0OE1/mix
         ture_k=50/model/model.pt already downloaded                                                               


/work/PRTNR/CHUV/DIR/rgottar1/spatial/env/jbac/miniforge3/envs/spatial/lib/python3.11/site-packages/spatialdata/_core/spatialdata.py:185: UserWarning: The table is annotating 'cell_labels', which is not present in the SpatialData object.
  self.validate_table_in_spatialdata(v)
/work/PRTNR/CHUV/DIR/rgottar1/spatial/env/jbac/miniforge3/envs/spatial/lib/python3.11/site-packages/lightning/fabric/plugins/environments/slurm.py:191: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /work/PRTNR/CHUV/DIR/rgottar1/spatial/env/jbac/minif ...
/work/PRTNR/CHUV/DIR/rgottar1/spatial/env/jbac/miniforge3/envs/spatial/lib/python3.11/site-packages/scanpy/neighbors/__init__.py:700: FutureWarning: `transformer='rapids'` is deprecated. Use `rapids_singlecell.tl.neighbors` instead.
  warn(msg, FutureWarning)
/work/PRTNR/CHUV/DIR/rgottar1/spatial/env/jbac/miniforge3/envs/spatial/lib

Epoch 1/50:   2%|▏         | 1/50 [00:00<00:03, 13.93it/s, v_num=1]

`Trainer.fit` stopped: `max_steps=1` reached.


Epoch 1/50:   2%|▏         | 1/50 [00:00<00:03, 13.37it/s, v_num=1]

/work/PRTNR/CHUV/DIR/rgottar1/spatial/env/jbac/miniforge3/envs/spatial/lib/python3.11/site-packages/lightning/fabric/plugins/environments/slurm.py:191: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /work/PRTNR/CHUV/DIR/rgottar1/spatial/env/jbac/minif ...



Sampling local variables, batch: 100%|██████████| 58/58 [00:48<00:00,  1.19it/s]


/work/PRTNR/CHUV/DIR/rgottar1/spatial/env/jbac/miniforge3/envs/spatial/lib/python3.11/site-packages/lightning/fabric/plugins/environments/slurm.py:191: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /work/PRTNR/CHUV/DIR/rgottar1/spatial/env/jbac/minif ...


Sampling local variables, batch: 100%|██████████| 58/58 [00:31<00:00,  1.84it/s]


# panel

In [2]:
class Args():
    panel = Path('/work/PRTNR/CHUV/DIR/rgottar1/spatial/env/xenium_paper/data/xenium/processed/segmentation/10x_0um/breast/breast')
    results_dir=Path('../scratch')
    xenium_processed_data_dir = Path(cfg['xenium_processed_data_dir'])
    cell_type_annotation_dir = Path(cfg['xenium_cell_type_annotation_dir'])
    segmentation='10x_0um'
    normalisation = 'lognorm'
    reference = 'matched_reference_combo'
    method = 'rctd_class_aware'
    level = 'Level2.1'
    min_counts=10
    min_features=5
    max_counts=float("inf")
    max_features=float("inf")
    min_cells=5
    use_batch=True
    l=None
    e=None
    mixture_k=50
    max_epochs=1
    batch_size=5000
    num_samples=1
    annotation_mode='reference_based'

args = Args()

In [3]:
import dask

dask.config.set({"dataframe.query-planning": False})

from pathlib import Path
import numpy as np
import pandas as pd
import scanpy as sc
import scvi
import torch
import argparse
import os
import sys

sys.path.append("workflow/scripts/")
import preprocessing
import readwrite
torch.set_float32_matmul_precision('medium')


# Set up argument parser
def parse_args():
    parser = argparse.ArgumentParser(description="Run RESOLVI on a Xenium sample.")
    parser.add_argument("--panel", type=Path, help="Path to the args.panel file.")
    parser.add_argument("--xenium_processed_data_dir", type=Path, help="Path to the xenium processed data directories")
    parser.add_argument("--cell_type_annotation_dir", type=Path, help="Path to the cell_type_annotation_dir.")
    parser.add_argument("--annotation_mode", type=str, help="annotation mode")
    parser.add_argument("--normalisation", type=str, help="annotation normalisation args.method")
    parser.add_argument("--reference", type=str, help="annotation reference")
    parser.add_argument("--method", type=str, help="annotation method")
    parser.add_argument("--level", type=str, help="annotation level")
    parser.add_argument("--out_dir_resolvi_model", type=str, help="output directory with RESOLVI model weights")
    parser.add_argument("--min_counts", type=int, help="QC parameter from pipeline config")
    parser.add_argument("--min_features", type=int, help="QC parameter from pipeline config")
    parser.add_argument("--max_counts", type=float, help="QC parameter from pipeline config")
    parser.add_argument("--max_features", type=float, help="QC parameter from pipeline config")
    parser.add_argument("--min_cells", type=int, help="QC parameter from pipeline config")
    parser.add_argument(
        "--max_epochs",
        type=int,
        default=50,
        help="Maximum number of epochs to train the model.",
    )
    parser.add_argument("--cell_type_labels", type=str, help="optional cell_type_labels for semi-supervised mode")
    parser.add_argument("--mixture_k", type=int, help="mixture_k parameter for unsupervised RESOLVI")
    parser.add_argument("--use_batch", action="store_true", help="whether to use batch parameter for RESOLVI")
    parser.add_argument(
        "-l",
        type=str,
        default=None,
        help="path to the log file",
    )

    ret = parser.parse_args()
    if not os.path.isdir(ret.path):
        raise RuntimeError(f"Error! Input directory does not exist: {ret.path}")
    if ret.max_epochs <= 0:
        ret.max_epochs = 50

    return ret


if __name__ == "__main__":
    # args = parse_args()

    if args.l is not None:
        old_stdout = sys.stdout
        old_stderr = sys.stderr

        _log = open(args.l, "w", encoding="utf-8")

        sys.stdout = _log
        sys.stderr = _log

    # process params
    if args.level is not None:
        print("Reading samples and cell type annotation")
        labels_key = "labels_key"
        semisupervised = True
    else:
        print("Reading samples")
        labels_key = None
        semisupervised = False

    if args.use_batch:
        batch_key = "sample"
        print(f"Using {batch_key=} for RESOLVI")
    else:
        batch_key = None

    condition = args.panel.parents[0].stem
    segmentation = args.panel.parents[1].stem

    # load data
    ads = {}
    for donor in (donors := args.panel.iterdir()):
        for sample in (samples_ := donor.iterdir()):
            print(donor.stem, sample.stem)

            if segmentation == "proseg_expected":
                k = ("proseg", condition, args.panel.stem, donor.stem, sample.stem)
                k_annot = (segmentation, condition, args.panel.stem, donor.stem, sample.stem)
                name = "/".join(k)
                name_annot = "/".join(k_annot)
                sample_dir = args.xenium_processed_data_dir / f"{name}/raw_results"
            else:
                k = (
                    segmentation.replace("proseg_mode", "proseg"),
                    condition,
                    args.panel.stem,
                    donor.stem,
                    sample.stem,
                )
                name = name_annot = "/".join(k)
                sample_dir = args.xenium_processed_data_dir / f"{name}/normalised_results/outs"

            ads[k] = readwrite.read_xenium_sample(sample_dir,anndata=True)
            ads[k].obs_names = ads[k].obs_names.astype(str)

            if segmentation == "proseg_expected":
                ads[k].obs_names = "proseg-" + ads[k].obs_names
                # need to round proseg expected counts for resolVI to run
                adata.X.data = adata.X.data.round()

            if args.level is not None:
                annot_file = (
                    args.cell_type_annotation_dir
                    / name_annot
                    / f"{args.normalisation}/reference_based/{args.reference}/{args.method}/{args.level}/single_cell/labels.parquet"
                )
                ads[k].obs[labels_key] = pd.read_parquet(annot_file).set_index("cell_id").iloc[:, 0]

    print("Concatenating")
    # concatenate
    xenium_levels = ["segmentation", "condition", "panel", "donor", "sample"]
    for k in ads.keys():
        for i, lvl in enumerate(xenium_levels):
            ads[k].obs[lvl] = k[i]
    adata = sc.concat(ads)
    adata = adata[adata.obs[labels_key].notna()].copy()
    print("Done")

    adata.X.data = adata.X.data.astype(np.float32)

    # preprocess (QC filters only)
    # resolvi requires at least 5 counts in each cell
    preprocessing.preprocess(
        adata,
        normalize=False,
        log1p=False,
        scale="none",
        pca=False,
        umap=False,
        save_raw=False,
        min_counts=args.min_counts,
        min_genes=args.min_features,
        max_counts=args.max_counts,
        max_genes=args.max_features,
        min_cells=args.min_cells,
        backend="cpu",
    )
    sc.pp.subsample(adata,n_obs=10000)
    scvi.external.RESOLVI.setup_anndata(
        adata, batch_key=batch_key, labels_key=labels_key, layer=None, prepare_data_kwargs={"spatial_rep": "spatial"}
    )
    resolvi = scvi.external.RESOLVI(adata, mixture_k=args.mixture_k, semisupervised=semisupervised)
    resolvi.train(max_epochs=args.max_epochs)
    resolvi.save(args.out_dir_resolvi_model, overwrite=True)

    if args.l is not None:
        _log.close()
        sys.stdout = old_stdout
        sys.stderr = old_stderr


Reading samples and cell type annotation
Using batch_key='sample' for RESOLVI
1FYB 1FYB
INFO     reading                                                                                                   
         /work/PRTNR/CHUV/DIR/rgottar1/spatial/env/xenium_paper/data/xenium/processed/segmentation/10x_0um/breast/b
         reast/1FYB/1FYB/normalised_results/outs/cell_feature_matrix.h5                                            


/work/PRTNR/CHUV/DIR/rgottar1/spatial/env/jbac/miniforge3/envs/spatial/lib/python3.13/site-packages/spatialdata/_core/spatialdata.py:158: UserWarning: The table is annotating 'cell_labels', which is not present in the SpatialData object.
  self.validate_table_in_spatialdata(v)


OPHI OPHI_3
INFO     reading                                                                                                   
         /work/PRTNR/CHUV/DIR/rgottar1/spatial/env/xenium_paper/data/xenium/processed/segmentation/10x_0um/breast/b
         reast/OPHI/OPHI_3/normalised_results/outs/cell_feature_matrix.h5                                          


/work/PRTNR/CHUV/DIR/rgottar1/spatial/env/jbac/miniforge3/envs/spatial/lib/python3.13/site-packages/spatialdata/_core/spatialdata.py:158: UserWarning: The table is annotating 'cell_labels', which is not present in the SpatialData object.
  self.validate_table_in_spatialdata(v)


OPHI OPHI_1
INFO     reading                                                                                                   
         /work/PRTNR/CHUV/DIR/rgottar1/spatial/env/xenium_paper/data/xenium/processed/segmentation/10x_0um/breast/b
         reast/OPHI/OPHI_1/normalised_results/outs/cell_feature_matrix.h5                                          


...

/work/PRTNR/CHUV/DIR/rgottar1/spatial/env/jbac/miniforge3/envs/spatial/lib/python3.13/site-packages/scvi/data/fields/_dataframe_field.py:187: UserWarning: Category 0 in adata.obs['_scvi_ind_x'] has fewer than 3 cells. Models may not train properly.
  categorical_mapping = _make_column_categorical(
/work/PRTNR/CHUV/DIR/rgottar1/spatial/env/jbac/miniforge3/envs/spatial/lib/python3.13/site-packages/lightning/fabric/plugins/environments/slurm.py:204: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /work/PRTNR/CHUV/DIR/rgottar1/spatial/env/jbac/minif ...
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/work/PRTNR/CHUV/DIR/rgottar1/spatial/env/jbac/miniforge3/envs/spatial/lib/python3.13/site-packages/lightning/fabric/plugins/environments/slurm.py:204: The `srun` command is available on your syste

Epoch 1/1: 100%|██████████| 1/1 [00:03<00:00,  3.50s/it, v_num=1]

`Trainer.fit` stopped: `max_epochs=1` reached.


Epoch 1/1: 100%|██████████| 1/1 [00:03<00:00,  3.52s/it, v_num=1]


AttributeError: 'Args' object has no attribute 'out_dir_resolvi_model'

In [ ]:
import dask

dask.config.set({"dataframe.query-planning": False})

import anndata as ad
import numpy as np
import pandas as pd
import scvi
import torch
import argparse
import os
import sys

sys.path.append("workflow/scripts/")
import preprocessing
import readwrite
torch.set_float32_matmul_precision('medium')


# Set up argument parser
def parse_args():
    parser = argparse.ArgumentParser(description="Embed panel of Xenium samples.")
    parser.add_argument("--path", type=str, help="Path to the xenium sample file.")
    parser.add_argument("--dir_resolvi_model", type=str, help="directory with saved RESOLVI model weights")
    parser.add_argument("--results_dir", type=str, help="results directory")
    parser.add_argument("--min_counts", type=int, help="QC parameter from pipeline config")
    parser.add_argument("--min_features", type=int, help="QC parameter from pipeline config")
    parser.add_argument("--max_counts", type=float, help="QC parameter from pipeline config")
    parser.add_argument("--max_features", type=float, help="QC parameter from pipeline config")
    parser.add_argument("--min_cells", type=int, help="QC parameter from pipeline config")
    parser.add_argument(
        "--num_samples",
        type=int,
        help="Number of samples for RESOLVI generative model.",
    )
    parser.add_argument("--batch_size", type=int, help="batch size parameter")
    parser.add_argument("--cell_type_labels", type=str, help="optional cell_type_labels for semi-supervised mode")
    parser.add_argument(
        "-l",
        type=str,
        default=None,
        help="path to the log file",
    )

    ret = parser.parse_args()
    if not os.path.isdir(ret.path):
        raise RuntimeError(f"Error! Input directory does not exist: {ret.path}")

    return ret


if __name__ == "__main__":
    # args = parse_args()

    if args.l is not None:
        old_stdout = sys.stdout
        old_stderr = sys.stderr

        _log = open(args.l, "w", encoding="utf-8")

        sys.stdout = _log
        sys.stderr = _log

    # process params
    if args.level is not None:
        print("Reading samples and cell type annotation")
        labels_key = "labels_key"
        semisupervised = True
    else:
        print("Reading samples")
        labels_key = None
        semisupervised = False

    if args.use_batch:
        batch_key = "sample"
        print(f"Using {batch_key=} for RESOLVI")
    else:
        batch_key = None

    condition = args.panel.parents[0].stem
    segmentation = args.panel.parents[1].stem

    # load data
    ads = {}
    for donor in (donors := args.panel.iterdir()):
        for sample in (samples_ := donor.iterdir()):
            print(donor.stem, sample.stem)

            if segmentation == "proseg_expected":
                k = ("proseg", condition, args.panel.stem, donor.stem, sample.stem)
                k_annot = (segmentation, condition, args.panel.stem, donor.stem, sample.stem)
                name = "/".join(k)
                name_annot = "/".join(k_annot)
                sample_dir = args.xenium_processed_data_dir / f"{name}/raw_results"
            else:
                k = (
                    segmentation.replace("proseg_mode", "proseg"),
                    condition,
                    args.panel.stem,
                    donor.stem,
                    sample.stem,
                )
                name = name_annot = "/".join(k)
                sample_dir = args.xenium_processed_data_dir / f"{name}/normalised_results/outs"

            ads[k] = readwrite.read_xenium_sample(sample_dir,anndata=True)
            ads[k].obs_names = ads[k].obs_names.astype(str)

            if segmentation == "proseg_expected":
                ads[k].obs_names = "proseg-" + ads[k].obs_names
                # need to round proseg expected counts for resolVI to run
                adata.X.data = adata.X.data.round()

            if args.level is not None:
                annot_file = (
                    args.cell_type_annotation_dir
                    / name_annot
                    / f"{args.normalisation}/{args.mode}/{args.reference}/{args.method}/{args.level}/single_cell/labels.parquet"
                )
                ads[k].obs[labels_key] = pd.read_parquet(annot_file).set_index("cell_id").iloc[:, 0]

    print("Concatenating")
    # concatenate
    xenium_levels = ["segmentation", "condition", "panel", "donor", "sample"]
    for k in ads.keys():
        for i, lvl in enumerate(xenium_levels):
            ads[k].obs[lvl] = k[i]
    adata = sc.concat(ads)
    adata = adata[adata.obs[labels_key].notna()].copy()
    print("Done")

    adata.X.data = adata.X.data.astype(np.float32)

    # preprocess (QC filters only)
    # resolvi requires at least 5 counts in each cell
    preprocessing.preprocess(
        adata,
        normalize=False,
        log1p=False,
        scale="none",
        pca=False,
        umap=False,
        save_raw=False,
        min_counts=args.min_counts,
        min_genes=args.min_features,
        max_counts=args.max_counts,
        max_genes=args.max_features,
        min_cells=args.min_cells,
        backend="cpu",
    )

    # resolvi = scvi.external.RESOLVI.load(args.dir_resolvi_model, adata=adata)

    samples_corr = resolvi.sample_posterior(
        model=resolvi.module.model_corrected,
        return_sites=["obs"],
        return_observed=True,
        summary_fun={"post_sample_q50": np.median},
        num_samples=args.num_samples,
        batch_size=args.batch_size,
        summary_frequency=100,
    )
    samples_corr = pd.DataFrame(samples_corr).T

    samples = resolvi.sample_posterior(
        model=resolvi.module.model_residuals,
        return_sites=["mixture_proportions"],
        summary_fun={"post_sample_means": np.mean},
        num_samples=args.num_samples,
        batch_size=args.batch_size,
        summary_frequency=100,
    )
    samples_proportions = pd.DataFrame(samples).T

    ### save
    samples_corr = pd.DataFrame(
        samples_corr.loc["post_sample_q50", "obs"],
        index=adata.obs_names,
        columns=adata.var_names,
    )

    proportions_cols = ["true_proportion", "diffusion_proportion", "background_proportion"]
    samples_proportions = pd.DataFrame(
        samples_proportions.loc["post_sample_means", "mixture_proportions"],
        index=adata.obs_names,
        columns=proportions_cols,
    )

    # add expression and metadata to new anndata
    adata_out = ad.AnnData(samples_corr)
    adata_out.obs = adata.obs
    for c in proportions_cols:
        adata_out.obs[c] = samples_proportions[c].values

    # save
    for _, (seg,cond,pan,don,samp) in adata_out.obs[xenium_levels].drop_duplicates().iterrows():

        k = (seg,cond,pan,don,samp,
             args.normalisation,args.annotation_mode,args.reference,args.method,args.level,
             f'mixture_k={args.mixture_k}',f'num_samples={args.num_samples}')
        name = '/'.join(k)

        out_file_resolvi_corrected_counts = args.results_dir / f'resolvi_supervised/{name}/corrected_counts.h5'
        out_file_resolvi_proportions = args.results_dir/ f'resolvi_supervised/{name}/proportions.parquet'

        # subset sample
        adata_out_sub = adata_out[(adata_out.obs[xenium_levels]==[seg,cond,pan,don,samp]).all(axis=1)].copy()

        # write
        readwrite.write_10X_h5(adata_out_sub, out_file_resolvi_corrected_counts)
        adata_out_sub.obs[proportions_cols].to_parquet(out_file_resolvi_proportions)

        del adata_out_sub

    if args.l is not None:
        _log.close()
        sys.stdout = old_stdout
        sys.stderr = old_stderr


Reading samples and cell type annotation
Using batch_key='sample' for RESOLVI
1FYB 1FYB
INFO     reading                                                                                                   
         /work/PRTNR/CHUV/DIR/rgottar1/spatial/env/xenium_paper/data/xenium/processed/segmentation/10x_0um/breast/b
         reast/1FYB/1FYB/normalised_results/outs/cell_feature_matrix.h5                                            
OPHI OPHI_3
INFO     reading                                                                                                   
         /work/PRTNR/CHUV/DIR/rgottar1/spatial/env/xenium_paper/data/xenium/processed/segmentation/10x_0um/breast/b
         reast/OPHI/OPHI_3/normalised_results/outs/cell_feature_matrix.h5                                          


/work/PRTNR/CHUV/DIR/rgottar1/spatial/env/jbac/miniforge3/envs/spatial/lib/python3.13/site-packages/spatialdata/_core/spatialdata.py:158: UserWarning: The table is annotating 'cell_labels', which is not present in the SpatialData object.
  self.validate_table_in_spatialdata(v)


OPHI OPHI_1
INFO     reading                                                                                                   
         /work/PRTNR/CHUV/DIR/rgottar1/spatial/env/xenium_paper/data/xenium/processed/segmentation/10x_0um/breast/b
         reast/OPHI/OPHI_1/normalised_results/outs/cell_feature_matrix.h5                                          


/work/PRTNR/CHUV/DIR/rgottar1/spatial/env/jbac/miniforge3/envs/spatial/lib/python3.13/site-packages/spatialdata/_core/spatialdata.py:158: UserWarning: The table is annotating 'cell_labels', which is not present in the SpatialData object.
  self.validate_table_in_spatialdata(v)


OPHI OPHI_2
INFO     reading                                                                                                   
         /work/PRTNR/CHUV/DIR/rgottar1/spatial/env/xenium_paper/data/xenium/processed/segmentation/10x_0um/breast/b
         reast/OPHI/OPHI_2/normalised_results/outs/cell_feature_matrix.h5                                          


...

Removed 0  cells...
Removed 0  genes...
Sampling local variables, batch: 100%|██████████| 291/291 [02:49<00:00,  1.71it/s]


/work/PRTNR/CHUV/DIR/rgottar1/spatial/env/jbac/miniforge3/envs/spatial/lib/python3.13/site-packages/lightning/fabric/plugins/environments/slurm.py:204: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /work/PRTNR/CHUV/DIR/rgottar1/spatial/env/jbac/minif ...


Sampling local variables, batch: 100%|██████████| 291/291 [02:34<00:00,  1.88it/s]


/work/PRTNR/CHUV/DIR/rgottar1/spatial/env/jbac/miniforge3/envs/spatial/lib/python3.13/site-packages/anndata/_core/anndata.py:1756: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


AttributeError: 'Args' object has no attribute 'out_file_resolvi_corrected_counts'

In [8]:
    samples_corr = resolvi.sample_posterior(
        model=resolvi.module.model_corrected,
        return_sites=["obs"],
        return_observed=True,
        summary_fun={"post_sample_q50": np.median},
        num_samples=args.num_samples,
        batch_size=args.batch_size,
        summary_frequency=100,
    )
    samples_corr = pd.DataFrame(samples_corr).T

    samples = resolvi.sample_posterior(
        model=resolvi.module.model_residuals,
        return_sites=["mixture_proportions"],
        summary_fun={"post_sample_means": np.mean},
        num_samples=args.num_samples,
        batch_size=args.batch_size,
        summary_frequency=100,
    )
    samples_proportions = pd.DataFrame(samples).T

    ### save
    samples_corr = pd.DataFrame(
        samples_corr.loc["post_sample_q50", "obs"],
        index=adata.obs_names,
        columns=adata.var_names,
    )

    proportions_cols = ["true_proportion", "diffusion_proportion", "background_proportion"]
    samples_proportions = pd.DataFrame(
        samples_proportions.loc["post_sample_means", "mixture_proportions"],
        index=adata.obs_names,
        columns=proportions_cols,
    )

    # add expression and metadata to new anndata
    adata_out = ad.AnnData(samples_corr)
    adata_out.obs = adata.obs
    for c in proportions_cols:
        adata_out.obs[c] = samples_proportions[c].values

    # save
    for _, (seg,cond,pan,don,samp) in adata_out.obs[xenium_levels].drop_duplicates().iterrows():

        k = (seg,cond,pan,don,samp,
             args.normalisation,args.annotation_mode,args.reference,args.method,args.level,
             f'mixture_k={args.mixture_k}',f'num_samples={args.num_samples}')
        name = '/'.join(k)

        out_file_resolvi_corrected_counts = args.results_dir / f'resolvi_supervised/{name}/corrected_counts.h5'
        out_file_resolvi_proportions = args.results_dir/ f'resolvi_supervised/{name}/proportions.parquet'

        print("saving to:",out_file_resolvi_corrected_counts,'\n',out_file_resolvi_proportions)

        # subset sample
        adata_out_sub = adata_out[(adata_out.obs[xenium_levels]==[seg,cond,pan,don,samp]).all(axis=1)].copy()

        # write
        out_file_resolvi_corrected_counts.parent.mkdir(exist_ok=True,parents=True)
        readwrite.write_10X_h5(adata_out_sub, out_file_resolvi_corrected_counts)
        adata_out_sub.obs[proportions_cols].to_parquet(out_file_resolvi_proportions)

        del adata_out_sub

/work/PRTNR/CHUV/DIR/rgottar1/spatial/env/jbac/miniforge3/envs/spatial/lib/python3.13/site-packages/lightning/fabric/plugins/environments/slurm.py:204: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /work/PRTNR/CHUV/DIR/rgottar1/spatial/env/jbac/minif ...


Sampling local variables, batch: 100%|██████████| 2/2 [00:01<00:00,  1.53it/s]


/work/PRTNR/CHUV/DIR/rgottar1/spatial/env/jbac/miniforge3/envs/spatial/lib/python3.13/site-packages/lightning/fabric/plugins/environments/slurm.py:204: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /work/PRTNR/CHUV/DIR/rgottar1/spatial/env/jbac/minif ...


Sampling local variables, batch: 100%|██████████| 2/2 [00:01<00:00,  1.53it/s]
saving to: ../scratch/resolvi_supervised/10x_0um/breast/breast/OPHI/OPHI_2/lognorm/reference_based/matched_reference_combo/rctd_class_aware/Level2.1/mixture_k=50/num_samples=1/corrected_counts.h5 
 ../scratch/resolvi_supervised/10x_0um/breast/breast/OPHI/OPHI_2/lognorm/reference_based/matched_reference_combo/rctd_class_aware/Level2.1/mixture_k=50/num_samples=1/proportions.parquet
saving to: ../scratch/resolvi_supervised/10x_0um/breast/breast/1GVR/1GVR/lognorm/reference_based/matched_reference_combo/rctd_class_aware/Level2.1/mixture_k=50/num_samples=1/corrected_counts.h5 
 ../scratch/resolvi_supervised/10x_0um/breast/breast/1GVR/1GVR/lognorm/reference_based/matched_reference_combo/rctd_class_aware/Level2.1/mixture_k=50/num_samples=1/proportions.parquet
saving to: ../scratch/resolvi_supervised/10x_0um/breast/breast/1FYB/1FYB/lognorm/reference_based/matched_reference_combo/rctd_class_aware/Level2.1/mixture_k=50